# Pulizia e integrazione dei dati

In [1]:
import pandas as pd
import statistics
import json
import requests
import time
import progressbar

In [2]:
# Si recuperano i dati aulla mobilità di Google.
mobilita_google_2020 = pd.read_csv('allegati/dataset/raw/mobilita-google-2020.csv')

In [3]:
# Si pulisce il dataset relativo alla mobilità. Si inizializza una variabile per accogliere gli indici delle righe da eliminare.
righe_da_eliminare = []

# Si inizializza la progressbar.
print('Eliminazione delle righe non necessarie.')
# Si blocca il sistema per allineare le print.
time.sleep(1)
with progressbar.ProgressBar(max_value=len(mobilita_google_2020)) as bar_eliminazione:
    # Per ogni riga...
    for riga in mobilita_google_2020.iterrows():
        # ... se il dato non è relativo alla media giornaliera della lombardia e la data non è nel periodo di riferimento, si aggiunge la riga a quelle da eliminare.
        if riga[1]['sub_region_1'] != 'Lombardy':
            righe_da_eliminare.append(riga[0])
        elif not pd.isnull(riga[1]['sub_region_2']):
            righe_da_eliminare.append(riga[0])
        elif riga[1]['date'].split('-')[1] in ['01', '02']:
            righe_da_eliminare.append(riga[0])
        # Si aggiorna la progressbar.
        bar_eliminazione.update(riga[0])
    
# Si eliminalo le righe.    
mobilita_google_2020 = mobilita_google_2020.drop(righe_da_eliminare)

# Si eliminano le colonne 'country_region_code', 'country_region', 'sub_region_1', 'sub_region_2', 'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id'.
mobilita_google_2020 = mobilita_google_2020.drop(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2', 'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id'], axis=1)

# Si salva il dataset pulito.
mobilita_google_2020.to_csv('allegati/dataset/mobilita-lombardia-2020.csv')

Eliminazione delle righe non necessarie.
100% (40764 of 40764) |##################| Elapsed Time: 0:00:07 Time:  0:00:07


In [4]:
# Si recupera il dataset parziale.
mobilita_google_2020 = pd.read_csv('allegati/dataset/mobilita-lombardia-2020.csv')

# Si inizializza una variabile contenete il dataset spacchettato.
dataset_spacchettato = pd.DataFrame(columns=['Data', 'Tipo', 'Valore'])

# Si inizializza una nuova progress bar per lo spacchettamento dei dati.
print('Spacchettamento dei dati.')
# Si blocca il sistema per allineare le print.
time.sleep(1)
with progressbar.ProgressBar(max_value=len(mobilita_google_2020)) as bar_spacchettamento:
    # Si inizializza una lista contenente i tipi delle misurazioni.
    tipi = ['retail_and_recreation', 'grocery_and_pharmacy', 'parks', 'transit_stations', 'workplaces', 'residential']
    suffisso = '_percent_change_from_baseline'

    # Per ogni riga del nuovo dataset...
    for indice, riga in zip(range(1, len(mobilita_google_2020)), mobilita_google_2020.iterrows()):
        # ... per ogni tipo di misurazione...
        for tipo in tipi:
            # ... si crea la nuova riga...
            nuova_riga = dict(Data=riga[1]['date'].replace('-', '/'), Tipo=tipo, Valore=riga[1][tipo + suffisso])
            # ... e la si inserisce nel dataset.
            dataset_spacchettato = dataset_spacchettato.append(nuova_riga, ignore_index=True)
        # Si aggiorna la progressbar.
        bar_spacchettamento.update(indice)

# Si salva il dataset pulito.
dataset_spacchettato.to_csv('allegati/dataset/mobilita-lombardia-2020.csv')

Spacchettamento dei dati.
100% (306 of 306) |######################| Elapsed Time: 0:00:06 Time:  0:00:06


Si crea ora un dataset per visualizzare i record su una mappa.

In [5]:
# Si recuperano i dati aulla mobilità di Google raw.
mobilita_mappa_google_2020 = pd.read_csv('allegati/dataset/raw/mobilita-google-2020.csv')

In [6]:
# Si pulisce il dataset relativo alla mobilità. Si inizializza una variabile per accogliere gli indici delle righe da eliminare.
righe_da_eliminare = []

# Si inizializza la progressbar.
print('Eliminazione delle righe non necessarie.')
# Si blocca il sistema per allineare le print.
time.sleep(1)
with progressbar.ProgressBar(max_value=len(mobilita_mappa_google_2020)) as bar_eliminazione:
    # Per ogni riga...
    for riga in mobilita_mappa_google_2020.iterrows():
        # ... se il dato non è relativo alla media giornaliera della lombardia e la data non è nel periodo di riferimento, si aggiunge la riga a quelle da eliminare.
        if riga[1]['sub_region_1'] != 'Lombardy':
            righe_da_eliminare.append(riga[0])
        elif pd.isnull(riga[1]['sub_region_2']):
            righe_da_eliminare.append(riga[0])
        elif riga[1]['date'].split('-')[1] in ['01', '02']:
            righe_da_eliminare.append(riga[0])
        # Si aggiorna la progressbar.
        bar_eliminazione.update(riga[0])
    
# Si eliminalo le righe.    
mobilita_mappa_google_2020 = mobilita_mappa_google_2020.drop(righe_da_eliminare)

# Si eliminano le colonne 'country_region_code', 'country_region', 'sub_region_1', 'sub_region_2', 'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id'.
mobilita_mappa_google_2020 = mobilita_mappa_google_2020.drop(['country_region_code', 'country_region', 'sub_region_1', 'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id'], axis=1)

# Si salva il dataset pulito.
mobilita_mappa_google_2020.to_csv('allegati/dataset/mobilita-mappa-lombardia-2020.csv')

Eliminazione delle righe non necessarie.
100% (40764 of 40764) |##################| Elapsed Time: 0:00:05 Time:  0:00:05


In [7]:
# Si recupera il dataset parziale.
mobilita_mappa_google_2020 = pd.read_csv('allegati/dataset/mobilita-mappa-lombardia-2020.csv')

# Si inizializza un dataset per la creazione della mappa.
mobilita_mappa = pd.DataFrame(columns=['Valore', 'Provincia'])

# Si inizializza una nuova progress bar per lo spacchettamento dei dati.
print('Spacchettamento dei dati.')
# Si blocca il sistema per allineare le print.
time.sleep(1)
with progressbar.ProgressBar(max_value=len(mobilita_mappa_google_2020)) as bar_spacchettamento:
    # Si inizializza una lista contenente i tipi delle misurazioni.
    tipi = ['retail_and_recreation', 'grocery_and_pharmacy', 'parks', 'transit_stations', 'workplaces', 'residential']
    suffisso = '_percent_change_from_baseline'

    # Per ogni riga del nuovo dataset...
    for indice, riga in zip(range(1, len(mobilita_mappa_google_2020)), mobilita_mappa_google_2020.iterrows()):
        # ... per ogni tipo di misurazione si sommano i valori...
        media = 0
        for tipo in tipi:
            # ... aggiunge il valore alla variabile.
            media += riga[1][tipo + suffisso]
        # Si calcola la media.
        media = media / len(tipi)

        # Se la media è NaN, si evita di aggiungere il record.
        if not pd.isnull(media) :
            # Si estrae il nome della provincia.
            provincia = ''
            if riga[1]['sub_region_2'] == 'Province of Monza and Brianza':
                provincia = 'Monza'
            else:
                provincia = riga[1]['sub_region_2'].split(' ')[-1]

            # Si aggiunge il record al dataset.
            nuova_riga = dict(Valore=media, Provincia=provincia)
            mobilita_mappa = mobilita_mappa.append(nuova_riga, ignore_index=True)

        # Si aggiorna la progressbar.
        bar_spacchettamento.update(indice)

# Si salva il dataset pulito.
mobilita_mappa.to_csv('allegati/dataset/mobilita-mappa-lombardia-2020.csv')

Spacchettamento dei dati.
100% (3672 of 3672) |####################| Elapsed Time: 0:00:10 Time:  0:00:10


In [8]:
# Si recupera il dataset parziale.
mobilita_mappa_google_2020 = pd.read_csv('allegati/dataset/mobilita-mappa-lombardia-2020.csv')

# Si inizializza un dataset per la creazione della mappa.
mobilita_mappa = pd.DataFrame(columns=['Valore', 'Provincia'])

# Si inizializza una lista contenente le province delle misurazioni.
province = ['Milan', 'Bergamo', 'Brescia', 'Como', 'Cremona', 'Lecco', 'Lodi', 'Mantua', 'Monza', 'Pavia', 'Sondrio', 'Varese']

# Si inizializza una nuova progress bar per la media delle province.
print('Calcolo medie per provincia.')
# Si blocca il sistema per allineare le print.
time.sleep(1)
with progressbar.ProgressBar(max_value=len(province)) as bar_medie:
    # Per ogni provincia nel dataset...
    for indice, provincia in zip(range(0, len(province)), province):
        # Si dichiara la variabile contenente la media.
        media_provincia = 0
        contatore_provincia = 0
        # Per ogni riga del nuovo dataset...
        for riga in mobilita_mappa_google_2020.iterrows():
            # ... per ogni tipo di misurazione si sommano i valori se della stessa provincia...
            if riga[1]['Provincia'] == provincia:
                # ... aggiunge il valore alla variabile e si incrementa il contatore.
                media_provincia += riga[1]['Valore']
                contatore_provincia += 1
        
        # Si calcola la media.
        media_provincia = media_provincia / contatore_provincia

        # Si aggiunge il record.
        nuova_riga = dict(Valore=media_provincia, Provincia=provincia)
        mobilita_mappa = mobilita_mappa.append(nuova_riga, ignore_index=True)

        # Si aggiorna la progressbar.
        bar_medie.update(indice)

# Si salva il dataset pulito.
mobilita_mappa.to_csv('allegati/dataset/mobilita-mappa-lombardia-2020.csv')

Calcolo medie per provincia.
100% (12 of 12) |########################| Elapsed Time: 0:00:03 Time:  0:00:03


In [9]:
# Si recupera il token pubblico di Mapbox.
token_mapbox = json.load(open('allegati/token-mapbox.json'))['token']

In [10]:
# Si recupera il dataset parziale.
mobilita_mappa_google_2020 = pd.read_csv('allegati/dataset/mobilita-mappa-lombardia-2020.csv')

# Si inizializza un dataset per la creazione della mappa con le coordinate.
mobilita_mappa = pd.DataFrame(columns=['Valore', 'Provincia', 'Latitudine', 'Longitudine'])

# Si inizializza una nuova progress bar per la media delle province.
print('Calcolo delle coordinate.')
# Si blocca il sistema per allineare le print.
time.sleep(1)
with progressbar.ProgressBar(max_value=len(province)) as bar_coordinate:
    # Per ogni riga del dataset...
    for indice, riga in zip(range(0, len(mobilita_mappa_google_2020)), mobilita_mappa_google_2020.iterrows()):
        # Si richiedono le coordinate.
        citta = '%20'.join(riga[1]['Provincia'].split(' '))
        coordinate = requests.get('https://api.mapbox.com/geocoding/v5/mapbox.places/' + citta + '%20Italy.json?access_token=' + token_mapbox).json()['features'][0]['center']
        
        # Si aggiunge la riga.
        nuova_riga = dict(Valore=riga[1]['Valore'], Provincia=riga[1]['Provincia'], Latitudine=coordinate[1], Longitudine=coordinate[0])
        mobilita_mappa = mobilita_mappa.append(nuova_riga, ignore_index=True)

        # Si incrementa la progressbar.
        bar_coordinate.update(indice)

# Si salva il dataset completo.
mobilita_mappa.to_csv('allegati/dataset/mobilita-mappa-lombardia-2020.csv')

Calcolo delle coordinate.
100% (12 of 12) |########################| Elapsed Time: 0:00:04 Time:  0:00:04
